<a href="https://colab.research.google.com/github/dinael2000/MDE_to_3D/blob/main/run_pipeline/full_workflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Leveraging Monocular Depth Estimation for the rapid 3D Digitization of Cultural Heritage Artefacts 🤖

## 💻 Full MDE-to-3D Pipeline

The following Notebook provides a full pipeline for the production of 3D models of coin-like objects, starting from a single image as input.

It is being developed as part of an ongoing Master Thesis on:
"Utilizing Monocular Depth Estimation (MDE) Models for the Digitization of Cultural Heritage Objects. Leveraging deep learning-based MDE-Algorithms for the production of 3D models of archaeological artefacts that carry micro-reliefs." carried out at the Universität zu Köln.  

### 1. Setting up the environment

In [1]:
%%capture
# 1.1. Clone github repository
!git clone https://github.com/dinael2000/MDE_to_3D.git

In [1]:
# 1.2.1. Install required libraries
%cd /content/MDE_to_3D

!pip install -r requirements.txt

/content/MDE_to_3D
  Using cached open3d-0.19.0-cp312-cp312-manylinux_2_31_x86_64.whl.metadata (4.3 kB)
  Using cached dash-3.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached configargparse-1.7.1-py3-none-any.whl.metadata (24 kB)
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached pyquaternion-0.9.9-py3-none-any.whl.metadata (1.4 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached comm-0.2.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 104.5 M

In [ ]:
# 1.2.2. Re-run after restarting session
!pip install -r requirements.txt
!pip install open3d

In [2]:
# 1.3. Initiate libraries
%%capture
import os

In [3]:
# 1.4. Read project modules
%cd /content/MDE_to_3D/run_pipeline

from depthanythingv1_function import *
from openback import *
from watertight import *
from merge_meshes import *
from scale_meshes import *
from utils_colors import *
from utils_pointcloud import *

/content/MDE_to_3D/run_pipeline


### 2. Set up depth estimation pipeline

In [4]:
# 2.1. Initiate directories

# Directory with input images
rgb_dir = "rgb_dir"
os.makedirs(rgb_dir, exist_ok=True)

# Directory to save .npy depth maps
depth_npy_dir = "depth_npy_dir"
os.makedirs(depth_npy_dir, exist_ok=True)

# Directory to save colored depth maps
depth_colored_dir = "depth_colored_dir"
os.makedirs(depth_colored_dir, exist_ok=True)

# Directory to save black-and-white depth maps
depth_bw_dir = "depth_bw_dir"
os.makedirs(depth_bw_dir, exist_ok=True)

# 2.2. Choose colormap-scheme for colored depth map
color_scheme = "inferno"

In [ ]:
# 2.3 Upload images
from google.colab import files

%cd /content/MDE_to_3D/run_pipeline/rgb_dir
uploads = files.upload()

%cd /content/MDE_to_3D/run_pipeline

/content/MDE_to_3D/run_pipeline/rgb_dir


Saving SB-443rev_001.jpg to SB-443rev_001.jpg
Saving SB-443obv_001.jpg to SB-443obv_001.jpg
/content/MDE_to_3D/run_pipeline


❗ Important Notes ❗

* Make sure that the object is mostly **centered** in the frame of the uploaded picture
* Make sure that the object occupies a **large space** in the frame of the uploaded picture
* Should you wish to *merge* two sides of an object (obv-rev), make sure that:
1. Each side occupies roughly the same space in each frame (to facilitate alignment)
2. Each side is named appropriately (e.g. "{object_name}-obv", "{object_name}-rev"






### 3. Depth Estimation with DepthAnything V1
Yang, L., Kang, B., Huang, Z., Xu, X., Feng, J., Zhao, H., "Depth Anything: Unleashing the Power of Large-Scale Unlabeled Data". 2024.

Available at: [GitHub](https://github.com/globalwetlands/depth-anything-V1), [HuggingFace](https://huggingface.co/spaces/LiheYoung/Depth-Anything)

In [ ]:
# Run depth estimation pipeline

run_depth_estimation(rgb_dir=rgb_dir, depth_npy_dir=depth_npy_dir, depth_bw_dir=depth_bw_dir, depth_colored_dir=depth_colored_dir, color_scheme=color_scheme)

Saved SB-443obv_001!
Saved SB-443rev_001!
Done!


### 4. 3D model creation

The following sections provide different options for creating 3D models, using the depth maps produced in the previous steps:

1. The *first* section enables the creation of non-watertight models, which can be used as such, or to then merge into a single model.

2. The *second* section enables the creation of (mostly) watertight models, which can be used as such.

3. The *third* section enables merging two (non-watertight) models into a single, full object.

4. The *fourth* section enables scaling any given model.


In [ ]:
# 4.1. Non-watertight models
out_dir_openback = "3d_models_openback"
os.makedirs(out_dir_openback, exist_ok=True)

# Parameters
pixel_size = 1.0 # recommended: 1.0
relief_scale = 7.0 # recommended: 6.0 - 9.0
poisson_depth = 9 # recommended: 8-9
output_format = "obj" # recommended: "obj", "ply"

batch_process_openback(depth_npy_dir, rgb_dir, out_dir_openback, output_format=output_format, pixel_size=pixel_size, relief_scale=relief_scale, poisson_depth=poisson_depth)

Processing SB-443obv_001
[Open3D WARNING] Write OBJ can not include triangle normals.
Saved 3d_models_openback/SB-443obv_001.obj
Processing SB-443rev_001
[Open3D WARNING] Write OBJ can not include triangle normals.
Saved 3d_models_openback/SB-443rev_001.obj


In [ ]:
# 4.2. Watertight models
out_dir_watertight = "3d_models_watertight"
os.makedirs(out_dir_watertight, exist_ok=True)

# Parameters
pixel_size = 1.0 # recommended: 1.0
relief_scale = 7.0 # recommended: 6.0 - 9.0
poisson_depth = 9 # recommended: 8-9
output_format = "obj" # recommended: "obj", "ply"

batch_process_watertight(depth_npy_dir, rgb_dir, out_dir_watertight, output_format=output_format, pixel_size=pixel_size, relief_scale=relief_scale, poisson_depth=poisson_depth)

Processing SB-443obv_001


In [ ]:
# 4.3. Merged models
input_dir = "3d_models_openback"
output_dir_merged = "3d_models_merged"
os.makedirs(output_dir_merged, exist_ok=True)

# Parameters
gap_factor = 0.0 # recommended: 0.0
scale = True
scale_factor = 0.25

batch_process_merge(input_dir, output_dir_merged, gap_factor=gap_factor, scale=scale, scale_factor=scale_factor)

In [ ]:
# 4.4. Scale models
input_dir_to_scale = "3d_models_merged"
output_dir_scaled = "3d_models_scaled"

# Parameters
scale_factor = 0.02

batch_scale_models(input_dir_to_scale, output_dir_scaled, scale_factor)